In [ ]:
# -*- coding: utf-8 -*-

import numpy as np
from vpython import *

scene = canvas()

        
#Class that will encapsulte the stress state (tensor) and also perform operations 
#to normalize and transform a given matrix
class StressState:
    
    #Constructur, takes in a stress tensor and the optional orientation matrix
    def __init__(self, stress_tensor, orientation_matrix = np.array([[1,0,0],[0,1,0],[0,0,1]])):
        
        #Initialize the stress tensor variable 
        self.stress_tensor = stress_tensor.copy()
        
        self.orientation_matrix = orientation_matrix.copy()
        
        #Transform the stress tensor, if for some reason the original orientation is not common to the x,y,z axis
        self.stress_transform(orientation_matrix.copy())
        

    
    #Method to perform a stress transofmation on the Stress State object
    #Requires the input of 'orientation matrix', which is an array of the N1,N2 & N3 vectors
    def stress_transform(self,orientation_matrix):

        original_orientation_matrix = self.orientation_matrix.copy()
        
        #First ensure that the orientation matrix has been normalized
        StressState.normalize_vector(orientation_matrix)
        
        #Set the class variable orientation to the (normalized) orientation matrix that was input from the user
        self.orientation_matrix = orientation_matrix.copy()
        
        
        #Placeholder variable for the transformed stress matrix
        transformed_state = np.zeros((3,3))
        
        
        #Variable that will be triggered if the axis of the transformation vectors are not all mutually perpendicular 
        perpendicular = True
        
        
        #A nested for loop will perform the stress transformations
        for i in range (3):  
            
            #Calculates the transformed stress component (sigma_P) on the new plane normal to N1, N2 or N3.
            #Follows the equation that Sigma_P = Transpose(Stress_Tensor) * N 
            sigma = self.stress_tensor.T@self.orientation_matrix[i]
            
            #Checks to see if the perpendicular variable has been set to false, will break out of the loop and
            #no stress transformation will occur            
            if perpendicular == False:
                    break
                
            #Performs the dot product operation to fill out the current row of the transformed stress matrix            
            for j in range (3):
                
                #Will cycle through N1, N2 & N3
                N = self.orientation_matrix[j]
                
                #Check to make sure the current normal verctor is perpendicular to the previous
                if ((round(np.inner(self.orientation_matrix[j-1],self.orientation_matrix[j]),2))) > 0.05:
                    
                    print('Stress transformation vectors are not mutually perpendicular\n'
                          + 'No transformation has been made, please try again')
                    perpendicular = False
                    self.orientation_matrix = original_orientation_matrix.copy()
                    break
                    
                #Set the current lacation in the rotated orientation to the correspodning value
                #By taking Sigma_P dotted with the normal vector for the corresponding column
                transformed_state[i,j] = round(np.inner(sigma,N),3)
        
        #If the perpendicular value was never set to false, perform the stress rotation on the StressState object     
        if perpendicular == True:
            self.stress_tensor = transformed_state.copy()    
        
        
    #Method to normalize any orientation matrix passed in 
    #Goes row by row to normalize N1, N2 & N3
    #The matrix is pass by reference, so the method does not need to return anything 
    #It will change the value of the original matrix passed in from outside the method 
    def normalize_vector(orientation_matrix):
        num_rows = np.shape(orientation_matrix)[0]
        num_cols = np.shape(orientation_matrix)[1]
        

        for i in range(num_rows):
            magnitude = np.linalg.norm(orientation_matrix[i])

            for j in range(num_cols):
                orientation_matrix[i][j] = round((orientation_matrix[i][j]/magnitude),4)
                
    #Method to plot a stress cube with the corresponding stress vector arrows displayed       
    def plot_cube(self, origin = [0,0,0]):
        

        #Set the position of the stress cube
        position = vector(origin[0],origin[1],origin[2])
        
        #Aligns the x axis of the stress cube with the N1 vector
        axis = vector(self.orientation_matrix[0][0],self.orientation_matrix[0][1],self.orientation_matrix[0][2])
        
        #Aligns the z axis (up) of the stress cube with the N3 vector
        up = vector(self.orientation_matrix[2][0],self.orientation_matrix[2][1],self.orientation_matrix[2][2])
        
        #Create the stress cube object and plot on the canvas
        box(pos=position, axis = axis, up = up, opacity=.5)

            
        #Nested for loop to create the arrows corresponding to the stresses on each face
        #Top level of loop will create the all of the arrows for a single direction
        for i in range(3):
            
            
            #The arrows will start on the outer face of the stress cube
            x_start = origin[0]+0.5*self.orientation_matrix[i][0]
            y_start = origin[1]+0.5*self.orientation_matrix[i][1]
            z_start = origin[2]+0.5*self.orientation_matrix[i][2]

            #Sets the arrow position (origin) variable
            arrow_position = vector(x_start,y_start,z_start)
         
            #Will create all arrows for a single direction
            for j in range(3):   
                
                
                x_vector = self.orientation_matrix[j][0]
                y_vector = self.orientation_matrix[j][1]
                z_vector = self.orientation_matrix[j][2]
                
                arrow_vector = vector(x_vector,y_vector,z_vector)
                
                arrow(pos=arrow_position, axis=arrow_vector, shaftwidth = .03, opacity=1, length = (.5))
                
                #Creates the labels for each arrow
                if (i<=j):
                    
                    #Offset the label from the arrow slightly 
                    x = x_start + 0.6*x_vector
                    y = y_start + 0.6*y_vector
                    z = z_start + 0.6*z_vector
                    label_position = vector(x,y,z)
                    
                    #Labels for the original orientation of the cube
                    axis_labels = ['x','y','z']
                    
                    #If the orientatio is not in the standard x, y, z coordinates, 
                    #Change labels to x', y' and z'
                    if not np.array_equal(self.orientation_matrix,np.array([[1,0,0],[0,1,0],[0,0,1]])):
                        axis_labels = ['x\'','y\'','z\'']
                    label_text = ('S_' + str(axis_labels[i]) + str(axis_labels[j]) + "= " + str(round(self.stress_tensor[i][j],2)))
                    label (pos = label_position, text = label_text, height = 8)
        
  
#Set the default values for the stress tensor
#Example is from pg. 28 of Chapter two notes   
stress_tensor = np.array([[100,50,-50],
                          [50,200,0],
                          [-50,0,200]])


#Set the default values for the transformation normal vectors
#Example is from pg. 28 of Chapter two notes 
n1 = np.array([.866,.25,.433])
n2 = np.array([-.5,.433,.75])
n3 = np.array([0,-.866,.5])
orientation_matrix = np.array([n1,n2,n3])


stress_cube = StressState(stress_tensor)
stress_cube.plot_cube()

#Initialize the lists that will be populated with the user inputs
stress_tensor_inputs = []
orientation_matrix_inputs = []


def text_test(s):
    pass

def input_button_pressed(b):
         
    print('Tranformed state shown!')
    for obj in scene.objects:
        obj.visible = False    

    for i in range(3):
        for j in range(3):
            stress_value = float(stress_tensor_inputs[i][j].text)
            orientation_value = float(orientation_matrix_inputs[i][j].text)            
            
            stress_tensor[i][j] = stress_value
            orientation_matrix[i][j] = orientation_value

    stress_cube = StressState(stress_tensor)
    stress_cube.plot_cube()
    stress_cube.stress_transform(orientation_matrix)
    stress_cube.plot_cube([2,0,0])



axis_labels = ['x','y','z']   
for i in range (3):
    stress_tensor_input_row = []
    for j in range (3):
        input_label = 'S_' + str(axis_labels[i]) + str(axis_labels[j]+ ': ')
        input_default = str(stress_tensor[i][j])
        input_label = wtext(text = input_label, pos = scene.title_anchor)
        input_text = winput(bind = text_test, width = 50,pos = scene.title_anchor, text = input_default)
        scene.append_to_title('  ')
        stress_tensor_input_row.append(input_text)
    scene.append_to_title('\n')
    stress_tensor_inputs.append(stress_tensor_input_row)


transformation_axis_labels = ['  l','m',' n']   
for i in range (3):
    orientation_matrix_input_row = []
    for j in range (3):
        input_label = str(transformation_axis_labels[j]) + str(i+1)+ ': '
        input_default = str(orientation_matrix[i][j])
        input_label = wtext(text = input_label, pos = scene.title_anchor)
        input_text = winput(bind = text_test, width = 50,pos = scene.title_anchor, text = input_default)
        scene.append_to_title('  ')
        orientation_matrix_input_row.append(input_text)
    scene.append_to_title('\n')
    orientation_matrix_inputs.append(orientation_matrix_input_row)

button(bind=input_button_pressed, text='Show stress transformation!' )
      